# Challenge 2: Sentiment Analysis

In this challenge we will learn sentiment analysis and practice performing sentiment analysis on Twitter tweets. 

## Introduction

Sentiment analysis is to *systematically identify, extract, quantify, and study affective states and subjective information* based on texts ([reference](https://en.wikipedia.org/wiki/Sentiment_analysis)). In simple words, it's to understand whether a person is happy or unhappy in producing the piece of text. Why we (or rather, companies) care about sentiment in texts? It's because by understanding the sentiments in texts, we will be able to know if our customers are happy or unhappy about our products and services. If they are unhappy, the subsequent action is to figure out what have caused the unhappiness and make improvements.

Basic sentiment analysis only understands the *positive* or *negative* (sometimes *neutral* too) polarities of the sentiment. More advanced sentiment analysis will also consider dimensions such as agreement, subjectivity, confidence, irony, and so on. In this challenge we will conduct the basic positive vs negative sentiment analysis based on real Twitter tweets.

NLTK comes with a [sentiment analysis package](https://www.nltk.org/api/nltk.sentiment.html). This package is great for dummies to perform sentiment analysis because it requires only the textual data to make predictions. For example:

```python
>>> from nltk.sentiment.vader import SentimentIntensityAnalyzer
>>> txt = "Ironhack is a Global Tech School ranked num 2 worldwide.   Our mission is to help people transform their careers and join a thriving community of tech professionals that love what they do."
>>> analyzer = SentimentIntensityAnalyzer()
>>> analyzer.polarity_scores(txt)
{'neg': 0.0, 'neu': 0.741, 'pos': 0.259, 'compound': 0.8442}
```

In this challenge, however, you will not use NLTK's sentiment analysis package because in your Machine Learning training in the past 2 weeks you have learned how to make predictions more accurate than that. The [tweets data](https://www.kaggle.com/kazanova/sentiment140) we will be using today are already coded for the positive/negative sentiment. You will be able to use the Naïve Bayes classifier you learned in the lesson to predict the sentiment of tweets based on the labels.

## Conducting Sentiment Analysis

### Loading and Exploring Data

The dataset we'll be using today is located in the lab directory named `Sentiment140.csv.zip`. You need to unzip it into a `.csv` file. Then in the cell below, load and explore the data.

*Notes:* 

* The dataset was downloaded from [Kaggle](https://www.kaggle.com/kazanova/sentiment140). We made a slight change on the original data so that each column has a label.

* The dataset is huuuuge (1.6m tweets). When you develop your data analysis codes, you can sample a subset of the data (e.g. 20k records) so that you will save a lot of time when you test your codes.

In [3]:
# your code here
#libaries
import pandas as pd
import re
import nltk
from nltk import word_tokenize
from nltk.stem import SnowballStemmer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords


In [4]:
# functions




def clean_up(s):
    first=re.sub("(http|https)://([^\s]+)", " ", s) #clean URL
    second=re.sub("\W", " " , first) # clean special character
    third= re.sub('\d', ' ',second) # clena numbers
    s_new=third.lower() # all lwoer case
    return s

def tokenize(s):
    tok=word_tokenize(s)
    return tok

def stem_and_lemmatize(l):
    new_list=[]
    for word in l:
        word_stemmed=stemmer.stem(word)
        word_lemmatized=lemmatizer.lemmatize(word_stemmed)
        new_list.append(word_lemmatized)
        
    return new_list


def remove_stopwords(l):
    newlist=[]
    for word in l:
        if word not in stopwords.words("english"):
            newlist.append(word)
    return newlist

def clean_it_all(s):
    s1= clean_up(s)
    s2=tokenize(s1)
    s3=stem_and_lemmatize(s2)
    s4=remove_stopwords(s3)
    
    return s4


In [5]:
stemmer=SnowballStemmer("english")
lemmatizer = WordNetLemmatizer()


In [6]:
#import data
data=pd.read_csv("../Sentiment140.csv")

### Prepare Textual Data for Sentiment Analysis

Now, apply the functions you have written in Challenge 1 to your whole data set. These functions include:

* `clean_up()`

* `tokenize()`

* `stem_and_lemmatize()`

* `remove_stopwords()`

Create a new column called `text_processed` in the dataframe to contain the processed data. At the end, your `text_processed` column should contain lists of word tokens that are cleaned up. Your data should look like below:

![Processed Data](data-cleaning-results.png)

In [7]:
# your code here
data.head()
data1=data[790000:810000]
data2=data[:200]
#data["text_processed"]=data.text.apply(lambda x: clean_it_all(x))

In [8]:
data1["text_processed"]=data1.text.apply(lambda x: clean_it_all(x))

/Users/Tilman/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [9]:
data1.head()
#text is not wel cleaned
#better now

,target,ids,date,flag,user,text,text_processed
790000,0,2325540224,Thu Jun 25 05:38:16 PDT 2009,NO_QUERY,Kain292,Virpi's last day. Busy as tits.,"[virpi, 's, last, day, ., busi, tit, .]"
790001,0,2325540629,Thu Jun 25 05:38:19 PDT 2009,NO_QUERY,dean_b,@SherilynMoon unfortunate then that I have no ...,"[@, sherilynmoon, unfortun, signal, tweet, pho..."
790002,0,2325540730,Thu Jun 25 05:38:20 PDT 2009,NO_QUERY,volleygalore,just dropped off the car to get the brakes loo...,"[drop, car, get, brake, look, ., hope, doe, n'..."
790003,0,2325540825,Thu Jun 25 05:38:20 PDT 2009,NO_QUERY,shOrtyyxx,Colin Kelly on Clyde1 Should get One Republic ...,"[colin, kelli, clyde1, get, one, republ, neyo,..."
790004,0,2325540910,Thu Jun 25 05:38:21 PDT 2009,NO_QUERY,DLPsYks,just a little upset i had a chance to take 4 o...,"[littl, upset, chanc, take, 4, 5, realli, cool..."


### Creating Bag of Words

The purpose of this step is to create a [bag of words](https://en.wikipedia.org/wiki/Bag-of-words_model) from the processed data. The bag of words contains all the unique words in your whole text body (a.k.a. *corpus*) with the number of occurrence of each word. It will allow you to understand which words are the most important features across the whole corpus.

Also, you can imagine you will have a massive set of words. The less important words (i.e. those of very low number of occurrence) do not contribute much to the sentiment. Therefore, you only need to use the most important words to build your feature set in the next step. In our case, we will use the top 5,000 words with the highest frequency to build the features.

In the cell below, combine all the words in `text_processed` and calculate the frequency distribution of all words. A convenient library to calculate the term frequency distribution is NLTK's `FreqDist` class ([documentation](https://www.nltk.org/api/nltk.html#module-nltk.probability)). Then select the top 5,000 words from the frequency distribution.

In [10]:
# your code here
from nltk.probability import FreqDist

In [13]:
wordbag=[]
for lists in data1.text_processed:
    for word in lists:
        wordbag.append(word)
    

In [14]:
freq=FreqDist(wordbag)

In [22]:
most=freq.most_common(5000)
most_word=most[0]
most=most[9:]
most

[('go', 1744),
 ('&', 1703),
 (':', 1671),
 ("'m", 1545),
 ('work', 1397),
 ('get', 1376),
 ('day', 1358),
 ('good', 1325),
 ('wa', 1311),
 ('today', 1055),
 ('like', 1052),
 ('-', 1013),
 ('http', 978),
 ('quot', 946),
 ('love', 840),
 ('time', 831),
 ('got', 821),
 ('one', 762),
 ('want', 740),
 ('u', 730),
 ('thank', 716),
 ('know', 711),
 ('think', 692),
 ('miss', 684),
 ('back', 671),
 ('feel', 639),
 ('morn', 634),
 ('lol', 598),
 ('realli', 590),
 ('ha', 581),
 ('need', 577),
 (')', 568),
 ('new', 560),
 ('ca', 557),
 ('hope', 556),
 ('amp', 556),
 ('see', 553),
 ('look', 549),
 ('well', 548),
 ('make', 543),
 ('sad', 542),
 ('(', 533),
 ('still', 526),
 ('twitter', 522),
 ('night', 504),
 ('im', 503),
 ('2', 492),
 ('last', 490),
 ('farrah', 490),
 ('oh', 475),
 ('come', 444),
 ('much', 443),
 ('home', 426),
 ('na', 422),
 ('great', 414),
 ('wish', 410),
 ('would', 381),
 ('watch', 381),
 ('tri', 379),
 ('sleep', 377),
 ('#', 375),
 ('could', 374),
 ('whi', 370),
 ('veri', 364)

In [18]:
#did clearly not clean well

### Building Features

Now let's build the features. Using the top 5,000 words, create a 2-dimensional matrix to record whether each of those words is contained in each document (tweet). Then you also have an output column to indicate whether the sentiment in each tweet is positive. For example, assuming your bag of words has 5 items (`['one', 'two', 'three', 'four', 'five']`) out of 4 documents (`['A', 'B', 'C', 'D']`), your feature set is essentially:

| Doc | one | two | three | four | five | is_positive |
|---|---|---|---|---|---|---|
| A | True | False | False | True | False | True |
| B | False | False | False | True | True | False |
| C | False | True | False | False | False | True |
| D | True | False | False | False | True | False|

However, because the `nltk.NaiveBayesClassifier.train` class we will use in the next step does not work with Pandas dataframe, the structure of your feature set should be converted to the Python list looking like below:

```python
[
	({
		'one': True,
		'two': False,
		'three': False,
		'four': True,
		'five': False
	}, True),
	({
		'one': False,
		'two': False,
		'three': False,
		'four': True,
		'five': True
	}, False),
	({
		'one': False,
		'two': True,
		'three': False,
		'four': False,
		'five': False
	}, True),
	({
		'one': True,
		'two': False,
		'three': False,
		'four': False,
		'five': True
	}, False)
]
```

To help you in this step, watch the [following video](https://www.youtube.com/watch?v=-vVskDsHcVc) to learn how to build the feature set with Python and NLTK. The source code in this video can be found [here](https://pythonprogramming.net/words-as-features-nltk-tutorial/).

[![Building Features](building-features.jpg)](https://www.youtube.com/watch?v=-vVskDsHcVc)

In [24]:
# your code here
#changed according to lab_review
feat=[]
for i,r in data1.iterrows():
    result={}
    for word in most:
        result[word]=(word in r["text_processed"])
        
    if r["target"]==0:
        sentiment=False
    else:
        sentiment=True
    feat.append((result,sentiment))

In [25]:
feat[0]

({('go', 1744): False,
  ('&', 1703): False,
  (':', 1671): False,
  ("'m", 1545): False,
  ('work', 1397): False,
  ('get', 1376): False,
  ('day', 1358): False,
  ('good', 1325): False,
  ('wa', 1311): False,
  ('today', 1055): False,
  ('like', 1052): False,
  ('-', 1013): False,
  ('http', 978): False,
  ('quot', 946): False,
  ('love', 840): False,
  ('time', 831): False,
  ('got', 821): False,
  ('one', 762): False,
  ('want', 740): False,
  ('u', 730): False,
  ('thank', 716): False,
  ('know', 711): False,
  ('think', 692): False,
  ('miss', 684): False,
  ('back', 671): False,
  ('feel', 639): False,
  ('morn', 634): False,
  ('lol', 598): False,
  ('realli', 590): False,
  ('ha', 581): False,
  ('need', 577): False,
  (')', 568): False,
  ('new', 560): False,
  ('ca', 557): False,
  ('hope', 556): False,
  ('amp', 556): False,
  ('see', 553): False,
  ('look', 549): False,
  ('well', 548): False,
  ('make', 543): False,
  ('sad', 542): False,
  ('(', 533): False,
  ('still', 

### Building and Traininng Naive Bayes Model

In this step you will split your feature set into a training and a test set. Then you will create a Bayes classifier instance using `nltk.NaiveBayesClassifier.train` ([example](https://www.nltk.org/book/ch06.html)) to train with the training dataset.

After training the model, call `classifier.show_most_informative_features()` to inspect the most important features. The output will look like:

```
Most Informative Features
	    snow = True            False : True   =     34.3 : 1.0
	  easter = True            False : True   =     26.2 : 1.0
	 headach = True            False : True   =     20.9 : 1.0
	    argh = True            False : True   =     17.6 : 1.0
	unfortun = True            False : True   =     16.9 : 1.0
	    jona = True             True : False  =     16.2 : 1.0
	     ach = True            False : True   =     14.9 : 1.0
	     sad = True            False : True   =     13.0 : 1.0
	  parent = True            False : True   =     12.9 : 1.0
	  spring = True            False : True   =     12.7 : 1.0
```

The [following video](https://www.youtube.com/watch?v=rISOsUaTrO4) will help you complete this step. The source code in this video can be found [here](https://pythonprogramming.net/naive-bayes-classifier-nltk-tutorial/).

[![Building and Training NB](nb-model-building.jpg)](https://www.youtube.com/watch?v=rISOsUaTrO4)

In [1]:
# your code here
import random
random.shuffle(feat)

train=feat[]

### Testing Naive Bayes Model

Now we'll test our classifier with the test dataset. This is done by calling `nltk.classify.accuracy(classifier, test)`.

As mentioned in one of the tutorial videos, a Naive Bayes model is considered OK if your accuracy score is over 0.6. If your accuracy score is over 0.7, you've done a great job!

In [ ]:
# your code here

## Bonus Question 1: Improve Model Performance

If you are still not exhausted so far and want to dig deeper, try to improve your classifier performance. There are many aspects you can dig into, for example:

* Improve stemming and lemmatization. Inspect your bag of words and the most important features. Are there any words you should furuther remove from analysis? You can append these words to further remove to the stop words list.

* Remember we only used the top 5,000 features to build model? Try using different numbers of top features. The bottom line is to use as few features as you can without compromising your model performance. The fewer features you select into your model, the faster your model is trained. Then you can use a larger sample size to improve your model accuracy score.

In [ ]:
# your code here

## Bonus Question 2: Machine Learning Pipeline

In a new Jupyter Notebook, combine all your codes into a function (or a class). Your new function will execute the complete machine learning pipeline job by receiving the dataset location and output the classifier. This will allow you to use your function to predict the sentiment of any tweet in real time. 

In [ ]:
# your code here

## Bonus Question 3: Apache Spark

If you have completed the Apache Spark advanced topic lab, what you can do is to migrate your pipeline from local to a Databricks Notebook. Share your notebook with your instructor and classmates to show off your achievements!

In [ ]:
# your code here